In [1]:
import numpy as np
import pandas as pd
import optuna
from sklearn.metrics import mean_squared_error


In [2]:
df = pd.read_csv('../../train_belgrade.csv')

df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [3]:

def snaive_forecasting(train_series, seasonal_lag):
    forecast = train_series.shift(seasonal_lag)
    return forecast


In [4]:

def objective(trial):
    # seasonal_lag = trial.suggest_int('seasonal_lag', 1, 288*7)
    seasonal_lag = 288

    train_size = int(len(df) * 0.9)
    train, test = df.iloc[:train_size], df.iloc[train_size:]

    forecast = snaive_forecasting(train['Detections'], seasonal_lag)[-len(test):]

    mse = mean_squared_error(test['Detections'], forecast)
    return mse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)


[I 2024-03-06 23:10:08,882] A new study created in memory with name: no-name-dd4edebc-0be9-4a70-94ab-6a1c5998cf8b
[I 2024-03-06 23:10:08,886] Trial 0 finished with value: 95.49112021857924 and parameters: {}. Best is trial 0 with value: 95.49112021857924.
[I 2024-03-06 23:10:08,890] Trial 1 finished with value: 95.49112021857924 and parameters: {}. Best is trial 0 with value: 95.49112021857924.
[I 2024-03-06 23:10:08,892] Trial 2 finished with value: 95.49112021857924 and parameters: {}. Best is trial 0 with value: 95.49112021857924.
[I 2024-03-06 23:10:08,894] Trial 3 finished with value: 95.49112021857924 and parameters: {}. Best is trial 0 with value: 95.49112021857924.
[I 2024-03-06 23:10:08,897] Trial 4 finished with value: 95.49112021857924 and parameters: {}. Best is trial 0 with value: 95.49112021857924.
[I 2024-03-06 23:10:08,899] Trial 5 finished with value: 95.49112021857924 and parameters: {}. Best is trial 0 with value: 95.49112021857924.
[I 2024-03-06 23:10:08,902] Trial 

In [5]:

# Best parameters and MSE
best_lag = study.best_params['seasonal_lag']
best_mse = study.best_value

print(f"Best Seasonal Lag: {best_lag}")
print(f"Best MSE: {best_mse}")


KeyError: 'seasonal_lag'

In [ ]:

train = df['Detections']
forecast = snaive_forecasting(train, best_lag)